# Experiment 079: Best-Work-Here Kernel Techniques (Rule-Compliant)

**Rationale**: Implement the key techniques from the best-work-here kernel in a rule-compliant way:
1. Non-linear mixture features: `A*(1-r) + B*r + 0.05*A*B*r*(1-r)`
2. Squeeze-and-Excitation blocks for feature recalibration
3. Residual blocks with LayerNorm and GELU
4. Adaptive ensemble: CatBoost + XGBoost + LightGBM + Neural Network
5. Advanced feature engineering (polynomial, interaction, statistical features)

**Rule Compliance**: The model class is self-contained and only requires changing the model definition line.

In [1]:
import sys
sys.path.append('/home/data/')

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

# Seed for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

print('Imports done')
print(f'GPU available: {torch.cuda.is_available()}')

Imports done
GPU available: True


In [2]:
# Local data loading functions
def load_data(data_type):
    if data_type == "single_solvent":
        df = pd.read_csv('/home/data/catechol_single_solvent_yields.csv')
        X = df[['Residence Time', 'Temperature', 'SOLVENT NAME']]
        Y = df[['SM', 'Product 2', 'Product 3']]
    elif data_type == "full":
        df = pd.read_csv('/home/data/catechol_full_data_yields.csv')
        X = df[['Residence Time', 'Temperature', 'SOLVENT A NAME', 'SOLVENT B NAME', 'SolventB%']]
        Y = df[['SM', 'Product 2', 'Product 3']]
    return X, Y

def load_features(feature_type):
    if feature_type == 'spange_descriptors':
        return pd.read_csv('/home/data/spange_descriptors_lookup.csv', index_col=0)
    elif feature_type == 'drfps':
        return pd.read_csv('/home/data/drfps_catechol_lookup.csv', index_col=0)
    elif feature_type == 'fragprints':
        return pd.read_csv('/home/data/fragprints_lookup.csv', index_col=0)
    elif feature_type == 'acs_pca_descriptors':
        return pd.read_csv('/home/data/acs_pca_descriptors_lookup.csv', index_col=0)

print('Data functions defined')

Data functions defined


In [3]:
# Official CV split functions (DO NOT MODIFY)
from typing import Any, Generator

def generate_leave_one_out_splits(
    X: pd.DataFrame, Y: pd.DataFrame
) -> Generator[
    tuple[tuple[pd.DataFrame, pd.DataFrame], tuple[pd.DataFrame, pd.DataFrame]],
    Any,
    None,
]:
    """Generate leave-one-out splits across the solvents."""
    for solvent in X["SOLVENT NAME"].unique():
        train_mask = X["SOLVENT NAME"] != solvent
        test_mask = X["SOLVENT NAME"] == solvent
        yield (
            (X[train_mask], Y[train_mask]),
            (X[test_mask], Y[test_mask]),
        )

def generate_leave_one_ramp_out_splits(
    X: pd.DataFrame, Y: pd.DataFrame
) -> Generator[
    tuple[tuple[pd.DataFrame, pd.DataFrame], tuple[pd.DataFrame, pd.DataFrame]],
    Any,
    None,
]:
    """Generate leave-one-ramp-out splits across the solvent ramps."""
    ramps = X["SOLVENT A NAME"].astype(str) + "_" + X["SOLVENT B NAME"].astype(str)
    for ramp in ramps.unique():
        train_mask = ramps != ramp
        test_mask = ramps == ramp
        yield (
            (X[train_mask], Y[train_mask]),
            (X[test_mask], Y[test_mask]),
        )

print('CV split functions defined')

CV split functions defined


In [4]:
# Squeeze-and-Excitation Block
class SEBlock(nn.Module):
    """Squeeze-and-Excitation block for feature recalibration"""
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(channels, max(channels // reduction, 4), bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(max(channels // reduction, 4), channels, bias=False),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return x * self.fc(x)

# Residual Block with SE attention
class ResidualBlock(nn.Module):
    """Enhanced residual block with SE attention"""
    def __init__(self, dim, dropout=0.3):
        super().__init__()
        self.block = nn.Sequential(
            nn.Linear(dim, dim),
            nn.LayerNorm(dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(dim, dim),
            nn.LayerNorm(dim),
        )
        self.se = SEBlock(dim)
        self.dropout = nn.Dropout(dropout)
        self.activation = nn.GELU()
    
    def forward(self, x):
        residual = x
        out = self.block(x)
        out = self.se(out)
        out = self.activation(residual + out)
        return self.dropout(out)

# Advanced Neural Network
class AdvancedNN(nn.Module):
    """Neural network with SE blocks and residual connections"""
    def __init__(self, input_dim, hidden_dims=[256, 128, 64], output_dim=3, dropout=0.3):
        super().__init__()
        
        layers = []
        # Input projection
        layers.extend([
            nn.Linear(input_dim, hidden_dims[0]),
            nn.LayerNorm(hidden_dims[0]),
            nn.GELU(),
            nn.Dropout(dropout)
        ])
        
        # Hidden layers with residual blocks
        for i in range(len(hidden_dims) - 1):
            layers.extend([
                nn.Linear(hidden_dims[i], hidden_dims[i+1]),
                nn.LayerNorm(hidden_dims[i+1]),
                nn.GELU(),
                nn.Dropout(dropout),
                ResidualBlock(hidden_dims[i+1], dropout)
            ])
        
        self.backbone = nn.Sequential(*layers)
        
        # Output head with sigmoid for [0,1] range
        self.output = nn.Sequential(
            nn.Linear(hidden_dims[-1], hidden_dims[-1] // 2),
            nn.GELU(),
            nn.Dropout(dropout * 0.5),
            nn.Linear(hidden_dims[-1] // 2, output_dim),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.backbone(x)
        return self.output(x)

print('Neural network architecture defined')

Neural network architecture defined


In [5]:
# Advanced Featurizer with non-linear mixture features
class AdvancedFeaturizer:
    """Featurizer with non-linear mixture features and advanced engineering"""
    
    def __init__(self, features='spange_descriptors', mixed=False):
        self.features_df = load_features(features)
        self.mixed = mixed
        self._cache = {}
        
    def _get_molecular(self, row):
        """Extract molecular features with caching"""
        if not self.mixed:
            key = row["SOLVENT NAME"]
            if key not in self._cache:
                self._cache[key] = self.features_df.loc[key].values
            return self._cache[key]
        else:
            A_name = row["SOLVENT A NAME"]
            B_name = row["SOLVENT B NAME"]
            r = row["SolventB%"] / 100.0  # Normalize to [0,1]
            
            if A_name not in self._cache:
                self._cache[A_name] = self.features_df.loc[A_name].values
            if B_name not in self._cache:
                self._cache[B_name] = self.features_df.loc[B_name].values
            
            A, B = self._cache[A_name], self._cache[B_name]
            # NON-LINEAR MIXING: Key technique from best-work-here kernel
            return A * (1 - r) + B * r + 0.05 * A * B * r * (1 - r)
    
    def _create_advanced_features(self, numeric_feat, mol_feat):
        """Engineer advanced features"""
        features = [numeric_feat, mol_feat]
        
        # Polynomial features
        if numeric_feat.shape[1] > 0:
            features.append(numeric_feat ** 2)
            features.append(np.sqrt(np.abs(numeric_feat) + 1e-8))
        
        # Interaction terms
        if numeric_feat.shape[1] >= 2:
            features.append((numeric_feat[:, 0] * numeric_feat[:, 1]).reshape(-1, 1))
        
        # Statistical features from molecular descriptors
        mol_stats = np.column_stack([
            mol_feat.mean(axis=1),
            mol_feat.std(axis=1),
            mol_feat.max(axis=1),
            mol_feat.min(axis=1)
        ])
        features.append(mol_stats)
        
        return np.concatenate(features, axis=1)
    
    def featurize(self, X):
        """Convert DataFrame to feature matrix"""
        if self.mixed:
            numeric_cols = ['Residence Time', 'Temperature', 'SolventB%']
        else:
            numeric_cols = ['Residence Time', 'Temperature']
        
        numeric = X[numeric_cols].values.astype(np.float32)
        mol = np.vstack([self._get_molecular(X.iloc[i]) for i in range(len(X))]).astype(np.float32)
        
        # Advanced feature engineering
        combined = self._create_advanced_features(numeric, mol)
        combined = np.nan_to_num(combined, nan=0.0, posinf=1e6, neginf=-1e6)
        
        return combined.astype(np.float32)

print('Advanced featurizer defined')

Advanced featurizer defined


In [6]:
# Best-Work-Here Model: Adaptive Ensemble
class BestWorkHereModel:
    """Adaptive ensemble with CatBoost, XGBoost, LightGBM, and Neural Network"""
    
    def __init__(self, data='single', hidden_dims=[256, 128, 64], dropout=0.3):
        self.data = data
        self.mixed = (data == 'full')
        self.featurizer = AdvancedFeaturizer('spange_descriptors', mixed=self.mixed)
        self.scaler = RobustScaler(quantile_range=(3, 97))
        
        # Will be set after featurization
        self.input_dim = None
        self.hidden_dims = hidden_dims
        self.dropout = dropout
        
        # Model weights (will be computed adaptively)
        self.weights = [0.25, 0.25, 0.25, 0.25]  # [xgb, lgb, rf, nn]
        
    def train_model(self, train_X, train_Y, num_epochs=150, lr=1e-3, batch_size=32):
        # Featurize
        X_np = self.featurizer.featurize(train_X)
        y_np = train_Y.values
        
        # Scale
        X_scaled = self.scaler.fit_transform(X_np)
        self.input_dim = X_scaled.shape[1]
        
        # Split for validation-based weight computation
        X_tr, X_val, y_tr, y_val = train_test_split(
            X_scaled, y_np, test_size=0.15, random_state=SEED
        )
        
        # Train XGBoost
        self.xgb = MultiOutputRegressor(xgb.XGBRegressor(
            n_estimators=200,
            max_depth=6,
            learning_rate=0.05,
            random_state=SEED,
            n_jobs=-1
        ))
        self.xgb.fit(X_tr, y_tr)
        xgb_val_pred = np.clip(self.xgb.predict(X_val), 0, 1)
        xgb_mse = np.mean((xgb_val_pred - y_val) ** 2)
        
        # Train LightGBM
        self.lgb = MultiOutputRegressor(lgb.LGBMRegressor(
            n_estimators=200,
            num_leaves=31,
            learning_rate=0.05,
            random_state=SEED,
            n_jobs=-1,
            verbose=-1
        ))
        self.lgb.fit(X_tr, y_tr)
        lgb_val_pred = np.clip(self.lgb.predict(X_val), 0, 1)
        lgb_mse = np.mean((lgb_val_pred - y_val) ** 2)
        
        # Train Random Forest
        self.rf = RandomForestRegressor(
            n_estimators=100,
            max_depth=10,
            random_state=SEED,
            n_jobs=-1
        )
        self.rf.fit(X_tr, y_tr)
        rf_val_pred = np.clip(self.rf.predict(X_val), 0, 1)
        rf_mse = np.mean((rf_val_pred - y_val) ** 2)
        
        # Train Neural Network
        self.nn = AdvancedNN(self.input_dim, self.hidden_dims, 3, self.dropout)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.nn.to(device)
        
        X_tensor = torch.tensor(X_tr, dtype=torch.float32)
        y_tensor = torch.tensor(y_tr, dtype=torch.float32)
        
        optimizer = torch.optim.AdamW(self.nn.parameters(), lr=lr, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
        criterion = nn.MSELoss()
        
        train_loader = DataLoader(
            TensorDataset(X_tensor, y_tensor),
            batch_size=batch_size, shuffle=True, drop_last=True
        )
        
        for epoch in range(num_epochs):
            self.nn.train()
            for inputs, targets in train_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                optimizer.zero_grad()
                loss = criterion(self.nn(inputs), targets)
                loss.backward()
                optimizer.step()
            scheduler.step()
        
        # Get NN validation predictions
        self.nn.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
            nn_val_pred = self.nn(X_val_tensor).cpu().numpy()
        nn_mse = np.mean((nn_val_pred - y_val) ** 2)
        
        # Compute adaptive weights (inverse MSE weighting)
        mses = np.array([xgb_mse, lgb_mse, rf_mse, nn_mse])
        inv_mses = 1.0 / (mses + 1e-8)
        self.weights = inv_mses / inv_mses.sum()
        
    def predict(self, test_X):
        X_np = self.featurizer.featurize(test_X)
        X_scaled = self.scaler.transform(X_np)
        
        # Get predictions from all models
        xgb_pred = np.clip(self.xgb.predict(X_scaled), 0, 1)
        lgb_pred = np.clip(self.lgb.predict(X_scaled), 0, 1)
        rf_pred = np.clip(self.rf.predict(X_scaled), 0, 1)
        
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.nn.eval()
        with torch.no_grad():
            X_tensor = torch.tensor(X_scaled, dtype=torch.float32).to(device)
            nn_pred = self.nn(X_tensor).cpu().numpy()
        
        # Weighted ensemble
        final_pred = (
            self.weights[0] * xgb_pred +
            self.weights[1] * lgb_pred +
            self.weights[2] * rf_pred +
            self.weights[3] * nn_pred
        )
        
        # Clip to [0, 1]
        final_pred = np.clip(final_pred, 0, 1)
        
        return torch.tensor(final_pred)

print('BestWorkHereModel defined')

BestWorkHereModel defined


In [ ]:
# Run CV for single solvent data
import tqdm

X, Y = load_data("single_solvent")
print(f"Single solvent data: {len(X)} samples, {len(X['SOLVENT NAME'].unique())} solvents")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []
fold_mses = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = BestWorkHereModel(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)
    
    predictions = model.predict(test_X)
    predictions_np = predictions.detach().cpu().numpy()
    
    # Calculate fold MSE
    fold_mse = np.mean((predictions_np - test_Y.values) ** 2)
    fold_mses.append(fold_mse)
    
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

submission_single_solvent = pd.DataFrame(all_predictions)
print(f"\nSingle solvent CV MSE: {np.mean(fold_mses):.6f} ± {np.std(fold_mses):.6f}")

In [ ]:
# Run CV for full (mixture) data
X, Y = load_data("full")
print(f"Full data: {len(X)} samples")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []
fold_mses = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = BestWorkHereModel(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)
    
    predictions = model.predict(test_X)
    predictions_np = predictions.detach().cpu().numpy()
    
    # Calculate fold MSE
    fold_mse = np.mean((predictions_np - test_Y.values) ** 2)
    fold_mses.append(fold_mse)
    
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

submission_full_data = pd.DataFrame(all_predictions)
print(f"\nFull data CV MSE: {np.mean(fold_mses):.6f} ± {np.std(fold_mses):.6f}")

In [ ]:
# Combine and save submission
submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"

print(f"Submission shape: {submission.shape}")
print(f"Columns: {submission.columns.tolist()}")

# Save
submission.to_csv("/home/submission/submission.csv", index=True)
print(f"\nSubmission saved to /home/submission/submission.csv")

# Verify
submission_check = pd.read_csv("/home/submission/submission.csv")
print(f"\nSubmission rows: {len(submission_check)}")
print(f"Expected: 656 (single) + 1227 (full) = 1883")

# Check prediction ranges
target_cols = ['target_1', 'target_2', 'target_3']
for col in target_cols:
    print(f"{col}: min={submission_check[col].min():.4f}, max={submission_check[col].max():.4f}")

In [ ]:
# Calculate overall CV score
single_mses = [0.011459, 0.009993, 0.006660, 0.016773, 0.016585]  # From exp_078 for comparison
# Use actual fold_mses from this run

print("="*50)
print("EXPERIMENT 079 COMPLETE")
print("="*50)
print(f"\nKey techniques implemented:")
print("1. Non-linear mixture features: A*(1-r) + B*r + 0.05*A*B*r*(1-r)")
print("2. Squeeze-and-Excitation blocks for feature recalibration")
print("3. Residual blocks with LayerNorm and GELU")
print("4. Adaptive ensemble: XGBoost + LightGBM + RF + Neural Network")
print("5. Advanced feature engineering (polynomial, interaction, statistical)")
print("\nThis is a RULE-COMPLIANT implementation that only changes the model definition.")